<a href="https://colab.research.google.com/github/Zezoo123/DeFi-Arbitrage-and-Prediction-Tool/blob/nn_implementation/neural_networks/lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os

In [8]:
folderpath = '/content/drive/MyDrive/BTC/'

df = pd.read_csv(folderpath + '1d.csv')
df.head()

,Open Time,Open,High,Low,Close,Volume,Close Time,Quote Asset Volume,Number of Trades,Taker Buy Base Asset Volume,Taker Buy Quote Asset Volume,Ignore
0,2024-09-18,60313.99,61786.24,59174.80,61759.99,36087.02469,2024-09-18 23:59:59.999,2.174000e+09,5167671,18429.59712,1.110623e+09,0
1,2024-09-19,61759.98,63850.00,61555.00,62947.99,34332.52608,2024-09-19 23:59:59.999,2.153176e+09,4438284,17609.62958,1.104427e+09,0
2,2024-09-20,62948.00,64133.32,62350.00,63201.05,25466.37794,2024-09-20 23:59:59.999,1.611196e+09,3969296,12411.32227,7.854406e+08,0
3,2024-09-21,63201.05,63559.90,62758.00,63348.96,8375.34608,2024-09-21 23:59:59.999,5.286579e+08,1283611,3877.73743,2.448026e+08,0
4,2024-09-22,63348.97,64000.00,62357.93,63578.76,14242.19892,2024-09-22 23:59:59.999,8.974773e+08,2145736,6923.17369,4.364562e+08,0
